In [ ]:
import os
from dotenv import load_dotenv
import sys
sys.path.append("../utils")

from twitter_api import search_recent_tweets
# Load .env file from project root
load_dotenv(dotenv_path="../.env")

# Get token securely
bearer_token = os.getenv("TWITTER_BEARER_TOKEN")
if not bearer_token:
    raise ValueError("Bearer token not found in .env")
else:
    query = "databricks OR spark lang:en -is:retweet"

tweets = search_recent_tweets(query, bearer_token)
print(tweets[0])  # Inspect one tweet
